In [1]:
import gzip
import json
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from scipy.sparse import lil_matrix
from torch_geometric.utils.convert import from_scipy_sparse_matrix
import torch
from pandas import json_normalize
from IPython.display import display

In [2]:
with gzip.open('../data/RosettaStone-GraphData-2023-09-22/superblue1/1/superblue1.json.gz','rb') as f:
    design = json.loads(f.read().decode('utf-8'))
    
instances = pd.DataFrame(design['instances'])
nets = pd.DataFrame(design['nets'])

conn=np.load('../data/RosettaStone-GraphData-2023-09-22/superblue1/superblue1_connectivity.npz')
A = coo_matrix((conn['data'], (conn['row'], conn['col'])), shape=conn['shape'])

with gzip.open('../data/RosettaStone-GraphData-2023-09-22/cells.json.gz', 'rb') as f:
    cells_json = json.loads(f.read())
cells = pd.DataFrame(cells_json).drop(columns=['name', 'width', 'height']).set_index('id')

In [3]:
cells['terms'] = cells['terms'].apply(lambda x: pd.DataFrame(x))
#cells

In [4]:
#instances.head()

In [5]:
A_tensor = torch.sparse_coo_tensor(
    torch.FloatTensor([A.row, A.col]),
    torch.Tensor(A.data)
).coalesce()

/var/folders/rg/mvn599q51g5_76hcm5b9cjgw0000gn/T/ipykernel_68365/1063470101.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1682343668887/work/torch/csrc/utils/tensor_new.cpp:248.)
  torch.FloatTensor([A.row, A.col]),


In [6]:
bad = []
global bad

In [7]:
# Get terminal index from cell type of instance
def direct(c, val):
    '''
    params:
    c: column index / terminal
    val: tensor value for lookup
    '''
    current_cell = cells['terms'][instances.iloc[c]['cell']]
    
    if val > current_cell.shape[0] - 1:
        bad.append((c, val))
        
    direction = current_cell.iloc[min(val, current_cell.shape[0] - 1)]['dir']
    return direction

In [8]:
# A_tensor.indices()[0] is columns of terminal matrix, or gates
indices = A_tensor.indices()
terminal_values = (A_tensor.values()).clone().detach()
gate_indices = indices[0]

In [9]:
out = lil_matrix((A_tensor.values().shape[0], 1))

In [10]:
i = 0
for gate_index, val_i in zip(gate_indices, range(terminal_values.shape[0])):
#     if i % 196595 == 0:
#         print(i)
   # i += 1
    out[val_i] = direct(c=int(gate_index.item()), val=int(terminal_values[val_i].item()) - 1)

In [11]:
# List of non-existent terminals
bad

[(732233, 54),
 (732245, 142),
 (732252, 144),
 (732533, 144),
 (732540, 146),
 (732547, 146),
 (732553, 146),
 (732560, 146),
 (732567, 146),
 (732573, 144),
 (732580, 146),
 (732588, 146),
 (732595, 146),
 (732601, 146),
 (732608, 146),
 (732615, 178),
 (732621, 180),
 (732628, 180),
 (732635, 180),
 (732641, 180),
 (732648, 180),
 (732655, 180),
 (732662, 180),
 (732668, 180),
 (732675, 180),
 (732683, 180),
 (732689, 180),
 (732703, 146),
 (732709, 146),
 (732716, 146),
 (732722, 146),
 (732729, 146),
 (732735, 146),
 (732742, 146),
 (732749, 146),
 (732755, 146),
 (732762, 146),
 (732769, 146),
 (732775, 146)]

In [12]:
#current cell
example = bad[0]
instance_ex = instances.iloc[example[0]]
terminal_list = cells['terms'][instance_ex['cell']]
print(f'Instance Id: {example[0]} \nTerminal Id: {example[1]}')
print('------')
print('Instance: ')
display(pd.DataFrame(instance_ex).T.set_index('id'))
print('------')
print('Terminals list: ')
display(terminal_list)

Instance Id: 732233 
Terminal Id: 54
------
Instance: 


,name,xloc,yloc,cell,orient
id,,,,,
732233,o765310,674285,1858140,20817,0


------
Terminals list: 


,name,id,dir,xloc,yloc
0,p0,1,1,9600,5920
1,p1,2,1,9600,85840
2,p2,3,1,9600,254560
3,p3,4,1,9600,113960
4,p4,5,1,9600,281200
5,p5,6,1,9600,226440
6,p6,7,1,9600,32560
7,p7,8,1,9600,59200
8,p8,9,1,9600,199800
9,p9,10,1,9600,213120


In [13]:
differences = []
for b in bad:
    diff = b[1] - (cells['terms'][instances.iloc[b[0]]['cell']].shape[0] - 1) # .iloc[b[1]]['dir']
    differences.append(diff)
differences

[2,
 1,
 3,
 1,
 3,
 3,
 3,
 3,
 3,
 1,
 3,
 3,
 3,
 3,
 3,
 1,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3]

In [14]:
out_tensor = torch.tensor(out.T.toarray())[0]

In [15]:
out_tensor

tensor([0., 0., 0.,  ..., 0., 0., 1.], dtype=torch.float64)

In [16]:
A_tensor.values().copy_(out_tensor)

tensor([0., 0., 0.,  ..., 0., 0., 1.])

In [17]:
 A_tensor

tensor(indices=tensor([[     0,      0,      0,  ..., 797937, 797937, 797937],
                       [613721, 613832, 613943,  ...,      0, 756354, 822927]]),
       values=tensor([0., 0., 0.,  ..., 0., 0., 1.]),
       size=(797938, 823024), nnz=2948927, layout=torch.sparse_coo)

In [18]:
#A = torch.sparse.mm(A_tensor, A_tensor.T)
A

<797938x823024 sparse matrix of type '<class 'numpy.int64'>'
	with 2950019 stored elements in COOrdinate format>